# Functional Test 3.1.3 - Attached Storage

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.1.3 for testing persistent storage attachment.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Request persistent storage at testing site

Through the portal project view click 'Request Storage' button (or ask Project Owner to do that). In the Jira ticket view request a single 10GB volume for the site that is being tested. Set `Needed until` 3 months forward. Name it the volume `acceptance-testing`. 

*Remember* that the storage must be requested on behalf of the 'FABRIC Maintenance' project. You must also be executing this notebook on behalf of this project.

## Step 3: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config()

## Step 4: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Create the test Slice

This creates a VM with a single attached volume on a specific worker at a specific site. 

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [ ]:
from datetime import datetime
from dateutil import tz

name='Node1'
storage_name = 'acceptance-testing'
site='MICH'
# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker=f'{site.lower()}-w1.fabric-testbed.net'
cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.3-Storage {site} on {worker} on {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=name, site=site, host=worker, cores=cores, ram=ram, disk=disk)
    
    node.add_storage(name=storage_name)

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice.show()
    slice.list_nodes()
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Format and mount the volume

The first time you use your persistent volume it will be a raw block device that needs to be formated.

In [ ]:
try:
    node = slice.get_node('Node1')
        
    storage = node.get_storage(storage_name)
    
    print(f"Storage Device Name: {storage.get_device_name()}")  
        
    stdout,stderr = node.execute(f"sudo mkfs.ext4 {storage.get_device_name()}")
    
except Exception as e:
    print(f"Exception: {e}")

### Mount the volume

Now the volume can be mounted.

In [ ]:
try:
        
    stdout,stderr = node.execute(f"sudo mkdir /mnt/fabric_storage; "
                                 f"sudo mount {storage.get_device_name()} /mnt/fabric_storage; "
                                 f"df -h")
    
    
except Exception as e:
    print(f"Exception: {e}")

## Step 8: Verify access

Verify that volume is read/writable.

In [ ]:
try:
    # create 1M file made of zeros
    stdout,stderr = node.execute(f"sudo dd if=/dev/zero of=/mnt/fabric_storage/zero-file bs=1024 count=1024")
    # check the file exists
    stdout,stderr = node.execute(f"ls -lh /mnt/fabric_storage")

except Exception as e:
    print(f"Exception: {e}")

## Step 9: Cleanup Your Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")